In [13]:
from langchain.llms import GooglePalm

In [19]:
api_key = "your_API_KEY"
llm = GooglePalm(google_api_key = api_key, temperature = 0.2)
poem = llm("Write a poem on google")
print(poem)

C:\Users\ssowmiya\Anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Google, oh Google,
You know everything.
From the weather to the news,
You've got me covered.

You're my search engine,
My friend, my guide.
You're always there for me,
No matter what.

I can't imagine my life without you,
Google.
You're the best thing that's ever happened to me.

Thank you for being there for me,
Google.
I love you.


In [21]:
from langchain.utilities import SQLDatabase

In [34]:
db_user = "root"
db_pw= ""
db_host = "localhost"
db_port = "3306"
db_name = "tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_pw}@{db_host}:{db_port}/{db_name}", sample_rows_in_table_info = 3)

In [39]:
from langchain_experimental.sql import SQLDatabaseChain

In [43]:
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose = True)

In [44]:
q1 = db_chain("How many white t-shirts available")

C:\Users\ssowmiya\Anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many white t-shirts available
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE color = 'White'
SQLResult: [(11,), (77,), (21,), (73,), (19,), (83,), (21,), (72,), (88,), (20,), (21,), (81,), (28,), (99,)]
Answer:11
> Finished chain.


In [45]:
q2 = db_chain("Can we get all small size tshirts")



> Entering new SQLDatabaseChain chain...
Can we get all small size tshirts
SQLQuery:SELECT * FROM t_shirts WHERE size = 'S'
SQLResult: [(4, 'Adidas', 'White', 'S', 38, 77), (6, 'Levi', 'Black', 'S', 42, 18), (9, 'Levi', 'Red', 'S', 40, 67), (10, 'Adidas', 'Black', 'S', 49, 72), (17, 'Van Huesen', 'Red', 'S', 37, 76), (20, 'Nike', 'White', 'S', 15, 19), (26, 'Nike', 'Black', 'S', 42, 14), (32, 'Adidas', 'Blue', 'S', 31, 99), (36, 'Van Huesen', 'Black', 'S', 14, 52), (46, 'Nike', 'Blue', 'S', 28, 55), (51, 'Adidas', 'Red', 'S', 31, 92), (70, 'Van Huesen', 'Blue', 'S', 13, 78), (88, 'Van Huesen', 'White', 'S', 26, 28)]
Answer:[4, 6, 9, 10, 17, 20, 26, 32, 36, 46, 51, 70, 88]
> Finished chain.


In [47]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "91"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "22292"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "16725.4"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "17462"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
     },
    {'Question': "how much sales amount will be generated if we sell all large size t shirts today in nike brand after discounts?",
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' and size="L"
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
    }
]

In [46]:
from langchain.embeddings import HuggingFaceEmbeddings

In [59]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
e = embeddings.embed_query("how many white t-shirt are there in levis brand?")

In [61]:
{ "How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'      Result of the SQL query,      91"},

({"How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'      Result of the SQL query,      91"},)

In [66]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [69]:
from langchain.vectorstores import Chroma

In [77]:
vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [84]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector =SemanticSimilarityExampleSelector(vectorstore = vectorstore, k=2,)


In [86]:
example_selector.select_examples({'Question': "How much is the total price of the inventory for all S-size t-shirts?"})

[{'Answer': '22292',
  'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '17462',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [94]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX,_mysql_prompt,MYSQL_PROMPT

In [100]:
print(MYSQL_PROMPT)

input_variables=['input', 'table_info', 'top_k'] template='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use CURDATE() function to get the current date, if the question involves "today".\n\nUse the following format:\n\nQues

In [102]:
from langchain.prompts.prompt import PromptTemplate
ex_prompt = PromptTemplate(inputformat = ["Question", "SQLQuery", "SQLResult", "Answer"],
              template = "\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}")


In [103]:
from langchain.prompts import FewShotPromptTemplate

In [106]:
fewshot_prompt= FewShotPromptTemplate(
example_selector = example_selector,example_prompt=ex_prompt,prefix=_mysql_prompt, suffix=PROMPT_SUFFIX, 
    input_variables=['input', 'table_info', 'top_k'],
)

In [107]:
new_chain = SQLDatabaseChain.from_llm(llm,db,verbose = True, prompt = fewshot_prompt)

In [115]:
q1 = new_chain("How many white levis shirts I have")



> Entering new SQLDatabaseChain chain...
How many white levis shirts I have
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(99,), (88,), (20,), (21,)]
Answer:20
> Finished chain.
